<a href="https://colab.research.google.com/github/ershadgithub/mmgdbm/blob/main/Ershad_of_3x3_Model_v4_both_attention_25_27_Exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/ITC_2025_GDBN/

/content/drive/MyDrive/ITC_2025_GDBN


In [ ]:
cd Experiment1/

/content/drive/MyDrive/ITC_2025_GDBN/Experiment1


In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch, random, os, cv2, argparse
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# from data_loader import getLoaders

In [ ]:
class Wafer_receptive_field_v1(Dataset):
    def __init__(self, data, param_data, label):
        self.data = data
        self.param_data = param_data
        self.label = label

    def __getitem__(self, index):
        return self.data[index], self.param_data[index], self.label[index]

    def __len__(self):
        return len(self.data)


class Wafer_receptive_field(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)



#############################################################
np.random.seed(42)
dir_path = os.getcwd()+"/"+"Wafer_3x3_25_27_Exp1"

X_test_dies_v1 =  np.load(dir_path+"/X_test_dies_v1.npy")
X_train_dies_v1 = np.load(dir_path+"/X_train_dies_v1.npy")
X_test_dies_param_v1 =  np.load(dir_path+"/X_test_dies_param_v1.npy")
X_train_dies_param_v1 = np.load(dir_path+"/X_train_dies_param_v1.npy")

labels = []
X_train_dies_v2 = []

# mu = 0.5965451703930418
# std = 0.3482061850943174

mu = 0
std = 1

# mu = 0.5424
# std = 0.30885
for x in X_train_dies_v1:
  labels.append(x[x.shape[0]//2, x.shape[1]//2].item())
  x1 = x.copy()
  x1 = (x1/x1.max() - mu)/std
  x1[x1.shape[0]//2, x1.shape[1]//2]=0
  X_train_dies_v2.append(x1)

wafer_tr_data = np.array(X_train_dies_v2)
labels_v1 = np.array(labels) - 1
y_train = labels_v1
print("Max Value wafer training data:", wafer_tr_data.max(), "Min Value wafer training data:",wafer_tr_data.min())


# Test set
y_test = []
X_test_dies_v2 = []
for x in X_test_dies_v1:
  y_test.append(x[x.shape[0]//2, x.shape[1]//2].item())
  x1 = x.copy()
  x1 = (x1/x1.max() - mu)/std
  x1[x1.shape[0]//2, x1.shape[1]//2]=0
  X_test_dies_v2.append(x1)

wafer_test_data = np.array(X_test_dies_v2)
wafer_test_label = np.array(y_test) - 1
wafer_test_label = torch.tensor(wafer_test_label).float()
print("Max Value wafer test data:", wafer_test_data.max(), "Min Value wafer test data:",wafer_test_data.min())
print()


# Test parametric data
# mus = [0.7762742501586967, 0.3104178934695622, 0.588879825610195, 0.5687342162150539, 0.4490443586054258, 0.37651138269966045, 0.4250408335662345]
# stds = [0.3490586951934936, 0.4072947386844294, 0.3516854901948171, 0.351697729393864, 0.3611352022022382, 0.3535069737222751, 0.3469982613766242]

mus = [0]*7
stds = [1]*7

# mus = [0.7342, 0.2377, 0.4723, 0.4366, 0.4012, 0.2977, 0.3469]
# stds = [0.31431, 0.37259, 0.27551, 0.26583, 0.32972, 0.30334, 0.29529]
X_train_dies_param_v2 = []
for test_param in X_train_dies_param_v1:
    test_params = []
    for ix,x in enumerate(test_param):
        x1 = x.copy()
        x1 = (x1/x1.max() - mus[ix])/stds[ix]
        x1[x1.shape[0]//2, x1.shape[1]//2]=0
        test_params.append(x1)
    X_train_dies_param_v2.append(test_params)
wafer_tr_param_data = np.array(X_train_dies_param_v2)

print("Max Value training parametric data:", wafer_tr_param_data.max(), "Min Value training parametric data:",wafer_tr_param_data.min())

# Test set
X_test_dies_param_v2 = []
for test_param in X_test_dies_param_v1:
    test_params = []
    for x in test_param:
        x1 = x.copy()
        x1 = (x1/x1.max() - mus[ix])/stds[ix]
        x1[x1.shape[0]//2, x1.shape[1]//2]=0
        test_params.append(x1)
    X_test_dies_param_v2.append(test_params)

wafer_test_param_data = np.array(X_test_dies_param_v2)
print("Max Value test parametric data:", wafer_test_param_data.max(), "Min Value test parametric data:",wafer_test_param_data.min())
print()

y_train_ran_index = np.random.permutation(len(y_train))
wafer_tr_data = wafer_tr_data[y_train_ran_index]
wafer_tr_param_data = wafer_tr_param_data[y_train_ran_index]
wafer_tr_label = np.array(y_train)[y_train_ran_index]
wafer_tr_label = torch.from_numpy(wafer_tr_label).float()


wafer_tr_param_data, wafer_val_param_data, _, _ = train_test_split(wafer_tr_param_data, wafer_tr_label, test_size=0.10, stratify=wafer_tr_label, random_state=42)
wafer_tr_data, wafer_val_data, wafer_tr_label, wafer_val_label = train_test_split(wafer_tr_data, wafer_tr_label, test_size=0.10, stratify=wafer_tr_label, random_state=42)

wafer_val_label = wafer_val_label.float()
wafer_val_data = torch.from_numpy(wafer_val_data)
wafer_val_data = wafer_val_data[:, None, :, :]

wafer_val_param_data = torch.from_numpy(wafer_val_param_data)
print("Validation data :", wafer_val_data.shape, "Validation test parametric data :", wafer_val_param_data.shape)

wafer_tr_data = torch.from_numpy(wafer_tr_data)
wafer_test_data = torch.from_numpy(wafer_test_data)
wafer_tr_data = wafer_tr_data[:, None, :, :]
wafer_test_data = wafer_test_data[:, None, :, :]

wafer_tr_param_data = torch.from_numpy(wafer_tr_param_data)
wafer_test_param_data = torch.from_numpy(wafer_test_param_data)

print("Training data :", wafer_tr_data.shape, "Test data :", wafer_test_data.shape)
print("Training parametric data :", wafer_tr_param_data.shape, "Test parametric data :", wafer_test_param_data.shape)
print()

Max Value wafer training data: 1.0 Min Value wafer training data: 0.0
Max Value wafer test data: 1.0 Min Value wafer test data: 0.0

Max Value training parametric data: 1.0 Min Value training parametric data: 0.0
Max Value test parametric data: 1.0 Min Value test parametric data: 0.0

Validation data : torch.Size([119583, 1, 3, 3]) Validation test parametric data : torch.Size([119583, 7, 3, 3])
Training data : torch.Size([1076247, 1, 3, 3]) Test data : torch.Size([297783, 1, 3, 3])
Training parametric data : torch.Size([1076247, 7, 3, 3]) Test parametric data : torch.Size([297783, 7, 3, 3])



In [ ]:
np.unique(y_train, return_counts=True), len(y_train)

((array([0, 1]), array([849976, 345854])), 1195830)

In [ ]:
np.unique(wafer_tr_label, return_counts=True), len(wafer_tr_label)

((array([0., 1.], dtype=float32), array([764978, 311269])), 1076247)

In [ ]:
np.unique(wafer_test_label, return_counts=True), len(wafer_test_label)

((array([0., 1.], dtype=float32), array([211613,  86170])), 297783)

In [ ]:
cls_cnts = np.unique(wafer_tr_label, return_counts=True)[1]
cls_cnts

array([764978, 311269])

In [ ]:

# train_loader, val_loader, test_loader = getLoaders(augmentation=False, batch_size=256, multiMode=True, data_path="Wafer_3x3_25_27_Exp1")

In [ ]:
def test_accuracy(network, test_loader, criterion, device, set_name, sl_thresh=0.5):
    """
    args:
    network: Wafer2Spike, an SNN model
    test_loader: Data loader for test set
    criterion: Crossentropy Loss
    device: CPU or CUDA (GPU)
    set_name: Val or Test set

    """

    test_loss = 0
    correct = 0
    network.eval()
    for batch_id, (data, param_data, target) in enumerate(test_loader):
        data, param_data, target = data.float().to(device), param_data.float().to(device), target.to(device)
        output = network(data, param_data)
        loss = criterion(output, target)
        # loss = ( (-target*torch.log(output + 1e-6))*pos_weight + (target-1)*torch.log(1-output + 1e-6) ).mean()
        test_loss += loss.to('cpu').item()
        correct += sum(np.where(output.data.cpu().numpy() >= sl_thresh, 1, 0) == target.data.cpu().numpy() )

    print("{} loss: {:.6f} | {} accuracy: {:.6f}".format(set_name, test_loss / len(test_loader), set_name, correct / len(test_loader.dataset)))



In [ ]:
def training(model, optimizer, criterion, device, sl_thresh=0.5, epochs=10, lr=0.0001, dataloaders=None, dropout_fc=0.3, hidden_neurons=128):

    """
    args:
    network: NN model
    epochs: Number of epochs for training
    lr: Learning rate
    dataloaders: Loading and preprocessing of data from a dataset into the training, validation, or testing pipelines
    numClasses: Number of classes
    dropout_fc: Dropout percentage for spiking-based fully connected layers

    """

    if len(dataloaders)==3:
        train_loader, val_loader, test_loader = dataloaders

    # Instantiating an optimizer
    # optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)


    # Training
    for e in range(epochs):
        loss_per_epoch = 0
        correct = 0
        model.train()
        for i, data in enumerate(train_loader):
            wafer_data, param_data, label = data
            wafer_data, param_data, label = wafer_data.float().to(device), param_data.float().to(device), label.to(device)
            optimizer.zero_grad()
            output = model(wafer_data, param_data)
            loss = criterion(output, label)

            loss.backward()
            optimizer.step()
            loss_per_epoch += loss.to('cpu').item()
            correct += sum(np.where(output.data.cpu().numpy() >= sl_thresh, 1, 0) == label.data.cpu().numpy() )

        # for param in decay_params:
        #     param.data = param.data.clamp(min=1e-7)

        print(f"Epoch: {e} | Training Loss: {loss_per_epoch / len(train_loader.dataset)} | Training accuracy : {correct / len(train_loader.dataset)}")

        if len(dataloaders)==3:
            test_accuracy(model, val_loader, criterion, device, "Validation", sl_thresh)
        elif len(dataloaders)==2:
            test_accuracy(model, test_loader, criterion, device, "Test", sl_thresh)


    pred = []
    y = []
    model.eval()
    for batch_id, (data, param_data, target) in enumerate(test_loader):
        data, param_data, target = data.float().to(device), param_data.float().to(device), target.to(device)
        pred += np.where(model(data, param_data).data.cpu().numpy() >= sl_thresh, 1, 0).tolist()
        y += target.data.cpu().numpy().tolist()

    pred = np.array(pred)
    y = np.array(y)
    # Confusion Matrix
    print()
    print("Confusion Matrix :")
    cf = confusion_matrix(y, pred)
    print(cf)
    print()
    print("Classification Report :")
    print(classification_report(y, pred))
    print()
    print("*"*25)

    # Defect_coverage: Ratio of defects covered by testing
    defect_coverage = 99.9/100 # Ratio of defects covered by testing
    # Number of negatives (negative dice): Dice testing bad
    N = np.where(y == 1)[0].shape[0]
    P = np.where(y == 0)[0].shape[0]
    test_escapes = ((1-defect_coverage)/defect_coverage) * N
    # NN: The number of the net-negative dice, i.e., the negative dice captured by Model with with SL > threshold
    index_1_y = np.where(np.array(y) == 1)[0]
    NN = (pred[index_1_y] == y[index_1_y]).sum()
    # Number of captured/covered test escapes
    CTE = (test_escapes/N)*NN
    # Yield Loss
    YL = cf[0][1].item()/cf.sum().item()

    """
    Rate for Return Merchandise Authorization (RMA) represents how many times of a single die's selling price equals the penalty of receiving a customer
    return. It implies the vendor would trade losing how many times of the die's test cost for removing one test escape. A GDBN model with a higher Gain is
    regarded as a better method.

    LOSS represents the total number of good dice discarded by the GDBN method.

    GAIN / Cost Reduction (CR) is the penalty to be paid minus the payment to be received if those test escapes are shipped to a customer, in terms of the
    times of a die's selling price.

    """

    RMA = 500
    LOSS = cf[0][1].item()
    GAIN = RMA * CTE - LOSS # CR
    DPPM = ((test_escapes - CTE)/ (P - LOSS - CTE)) * 10**6

    print("RMA: ", RMA)
    print("Defect_coverage :", defect_coverage*100, "%")
    print("Total number of negative dies :", N)
    print("Test Escapes :", np.round(test_escapes, 1))
    print("Covered Test Escapes :", CTE)
    print("Yield Loss :", YL)
    print("LOSS: ", LOSS)
    print("GAIN/CR: ", GAIN)
    print("DPPM: ", DPPM)
    print("*"*25)

In [ ]:
class AttentionBasedCNN(nn.Module):
  def __init__(self, in_channels=1, numHeads= 2, numCls=2, out_channels=32, dropout_fc=0.3, hidden_neurons=128):
    super(AttentionBasedCNN, self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.numCls = numCls
    self.dropout_fc = dropout_fc
    self.inter_channels = max(1, numHeads // 8)
    self.query_conv = nn.Conv2d(in_channels, self.inter_channels, kernel_size=1, bias=False)
    self.key_conv   = nn.Conv2d(in_channels, self.inter_channels, kernel_size=1, bias=False)
    self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1, bias=False)
    # self.gamma = nn.Parameter(torch.ones(1))

    self.conv1 = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=1)
    self.bn1 = nn.BatchNorm2d(self.out_channels)
    self.conv2 = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=1, stride=1)
    self.bn2 = nn.BatchNorm2d(self.out_channels)

    # To deal with Test parametric data
    self.query_conv_param = nn.Conv2d(7, self.inter_channels, kernel_size=1, bias=False)
    self.key_conv_param   = nn.Conv2d(7, self.inter_channels, kernel_size=1, bias=False)
    self.value_conv_param = nn.Conv2d(7, 7, kernel_size=1, bias=False)

    self.conv1_param = nn.Conv2d(7, self.out_channels, kernel_size=1, stride=1, groups=1)
    self.bn1_param = nn.BatchNorm2d(self.out_channels)
    self.conv2_param = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=1, stride=1)
    self.bn2_param = nn.BatchNorm2d(self.out_channels)

    self.fully_layers = nn.Sequential(nn.Linear(self.out_channels*3*3*2, hidden_neurons), nn.ReLU(inplace=True), nn.Dropout(p=self.dropout_fc), \
                                      nn.Linear(hidden_neurons, numCls), nn.Sigmoid())

  def forward(self, x, x_param):
    batch_size, C, height, width = x.size()
    _, C_param, _, _ = x_param.size()
    # Generate query, key, and value projections
    proj_query = self.query_conv(x).view(batch_size, self.inter_channels, -1)  # (B, inter_channels, H*W)
    proj_query = proj_query.permute(0, 2, 1)  # (B, H*W, inter_channels) (64,9,4)
    proj_key   = self.key_conv(x).view(batch_size, self.inter_channels, -1)    # (B, inter_channels, H*W) (64,4,9)

    # Compute the dot-product attention map 36 pixel values
    energy = torch.bmm(proj_query, proj_key)  # (B, H*W, H*W) attention weight (64, 9, 9) pairwise relation
    attention = F.softmax(energy, dim=-1)     # Normalize along the last dimension

    # Project the values
    proj_value = self.value_conv(x).view(batch_size, C, -1)  # (B, C, H*W) (64, 1, 9 )

    # Apply attention: weighted sum of values
    out = torch.bmm(proj_value, attention.permute(0, 2, 1))  # (B, C, H*W) (9,9 . 9,1 => 9,1)
    out = out.view(batch_size, C, height, width)

    # Combine with the input (residual connection)
    # out = self.gamma * out + x
    out = F.relu(self.bn1(self.conv1(out)))
    out = F.relu(self.bn2(self.conv2(out)))


    # Test Paramteric Data
    proj_query_param = self.query_conv_param(x_param).view(batch_size, self.inter_channels, -1)
    proj_query_param = proj_query_param.permute(0, 2, 1)
    proj_key_param = self.key_conv_param(x_param).view(batch_size, self.inter_channels, -1)

    # Compute the dot-product attention map
    energy_param = torch.bmm(proj_query_param, proj_key_param)
    attention_param = F.softmax(energy_param, dim=-1)

    # Project the values
    proj_value_param = self.value_conv_param(x_param).view(batch_size, C_param, -1)

    # Apply attention: weighted sum of values
    out_param = torch.bmm(proj_value_param, attention_param.permute(0, 2, 1))  # (B, C, H*W)
    out_param = out_param.view(batch_size, C_param, height, width)

    out_param = F.relu(self.bn1_param(self.conv1_param(out_param)))
    out_param = F.relu(self.bn2_param(self.conv2_param(out_param)))

    out_param = out_param.view(-1, self.out_channels*3*3)


    # print("before", out.shape)
    out = out.view(-1, self.out_channels*3*3)
    # print("after", out.shape)
    out_concat = torch.cat((out, out_param), dim=1)
    out = self.fully_layers(out_concat)
    return out.flatten()

def seed_worker(worker_id):
    np.random.seed(seed_num)
    random.seed(seed_num)



In [ ]:
batch_size = 256
dropout_fc = 0.35 # 0.25
hidden_neurons = 256
out_channels = 24
epochs = 14, 20, 23
lr = 0.001 # 0.001
sl_thresh = 0.5 # 0.5
model = AttentionBasedCNN(in_channels=1, numHeads=32, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)


In [ ]:
for w in model.state_dict().keys():
  print(w, model.state_dict()[w].shape)

query_conv.weight torch.Size([4, 1, 1, 1])
key_conv.weight torch.Size([4, 1, 1, 1])
value_conv.weight torch.Size([1, 1, 1, 1])
conv1.weight torch.Size([24, 1, 1, 1])
conv1.bias torch.Size([24])
bn1.weight torch.Size([24])
bn1.bias torch.Size([24])
bn1.running_mean torch.Size([24])
bn1.running_var torch.Size([24])
bn1.num_batches_tracked torch.Size([])
conv2.weight torch.Size([24, 24, 1, 1])
conv2.bias torch.Size([24])
bn2.weight torch.Size([24])
bn2.bias torch.Size([24])
bn2.running_mean torch.Size([24])
bn2.running_var torch.Size([24])
bn2.num_batches_tracked torch.Size([])
query_conv_param.weight torch.Size([4, 7, 1, 1])
key_conv_param.weight torch.Size([4, 7, 1, 1])
value_conv_param.weight torch.Size([7, 7, 1, 1])
conv1_param.weight torch.Size([24, 7, 1, 1])
conv1_param.bias torch.Size([24])
bn1_param.weight torch.Size([24])
bn1_param.bias torch.Size([24])
bn1_param.running_mean torch.Size([24])
bn1_param.running_var torch.Size([24])
bn1_param.num_batches_tracked torch.Size([])
conv

In [ ]:
# params = 0
# for p in model.parameters():
#   params += p.numel()
# params

In [ ]:
# # Save the entire model
# torch.save(model, 'attention_based_multi_model.pth')

In [ ]:
if __name__ == "__main__":

  for e in range(41, 7, -1):
    batch_size = 256
    dropout_fc = 0.35 # 0.25
    hidden_neurons = 256
    out_channels = 24
    epochs = e # 14, 20, 23
    lr = 0.001 # 0.001
    sl_thresh = 0.5 # 0.5

    # For reproducibility
    seed_num = 42
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    g = torch.Generator()
    g.manual_seed(seed_num)

    train_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_tr_data, wafer_tr_param_data, wafer_tr_label), batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_val_data, wafer_val_param_data, wafer_val_label), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_test_data, wafer_test_param_data, wafer_test_label), batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pos_weight = torch.tensor([(cls_cnts[0]/cls_cnts[1]).item()]).to(device)  # Actual Training ratio of classes after splitting (764978/311269)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    model = AttentionBasedCNN(in_channels=1, numHeads=32, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    training(model, optimizer, criterion, device, sl_thresh=sl_thresh, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)



Epoch: 0 | Training Loss: 0.003345499892103157 | Training accuracy : 0.8286350623973865
Validation loss: 0.988904 | Validation accuracy: 0.561284
Epoch: 1 | Training Loss: 0.003258267531188632 | Training accuracy : 0.8558853125722998
Validation loss: 0.829136 | Validation accuracy: 0.858349
Epoch: 2 | Training Loss: 0.0032223745666409985 | Training accuracy : 0.8681640924434633
Validation loss: 0.824318 | Validation accuracy: 0.869513
Epoch: 3 | Training Loss: 0.0032060811462772134 | Training accuracy : 0.873901622954582
Validation loss: 0.820630 | Validation accuracy: 0.875275
Epoch: 4 | Training Loss: 0.003198347015030633 | Training accuracy : 0.8763099920371439
Validation loss: 0.827318 | Validation accuracy: 0.873435
Epoch: 5 | Training Loss: 0.003191968386847758 | Training accuracy : 0.8784897890539997
Validation loss: 0.821170 | Validation accuracy: 0.878712
Epoch: 6 | Training Loss: 0.0031877563533622734 | Training accuracy : 0.8800619188717832
Validation loss: 0.824051 | Valida

In [ ]:
# dropout_fc = 0.28

Epoch: 0 | Training Loss: 0.0033314823390989718 | Training accuracy : 0.8321630629400129
Validation loss: 0.842016 | Validation accuracy: 0.854344
Epoch: 1 | Training Loss: 0.0032403588658620718 | Training accuracy : 0.8620335294778987
Validation loss: 0.835685 | Validation accuracy: 0.841942
Epoch: 2 | Training Loss: 0.0032119175241994377 | Training accuracy : 0.8720702589647172
Validation loss: 0.822946 | Validation accuracy: 0.876462
Epoch: 3 | Training Loss: 0.003199943632444661 | Training accuracy : 0.87599779604496
Validation loss: 0.818081 | Validation accuracy: 0.874547
Epoch: 4 | Training Loss: 0.0031930954921402495 | Training accuracy : 0.8782129009418842
Validation loss: 0.826214 | Validation accuracy: 0.860198
Epoch: 5 | Training Loss: 0.0031878315329587474 | Training accuracy : 0.8800275401464533
Validation loss: 0.818754 | Validation accuracy: 0.882701
Epoch: 6 | Training Loss: 0.0031846507129889992 | Training accuracy : 0.8812939780552234
Validation loss: 0.816147 | Vali

In [ ]:
if __name__ == "__main__":

  for e in range(41, 7, -1):
    batch_size = 256
    dropout_fc = 0.27 # 0.25
    hidden_neurons = 256
    out_channels = 24
    epochs = e # 14, 20, 23
    lr = 0.001 # 0.001
    sl_thresh = 0.5 # 0.5

    # For reproducibility
    seed_num = 42
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    g = torch.Generator()
    g.manual_seed(seed_num)

    train_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_tr_data, wafer_tr_param_data, wafer_tr_label), batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_val_data, wafer_val_param_data, wafer_val_label), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_test_data, wafer_test_param_data, wafer_test_label), batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pos_weight = torch.tensor([(cls_cnts[0]/cls_cnts[1]).item()]).to(device)  # Actual Training ratio of classes after splitting (764978/311269)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    model = AttentionBasedCNN(in_channels=1, numHeads=32, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    training(model, optimizer, criterion, device, sl_thresh=sl_thresh, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)



Epoch: 0 | Training Loss: 0.0033441028855381006 | Training accuracy : 0.8290494654108211
Validation loss: 0.861932 | Validation accuracy: 0.794135
Epoch: 1 | Training Loss: 0.0032878640939476736 | Training accuracy : 0.8469849393308413
Validation loss: 0.831780 | Validation accuracy: 0.859888
Epoch: 2 | Training Loss: 0.003234404103564265 | Training accuracy : 0.8640033375238212
Validation loss: 0.827913 | Validation accuracy: 0.872072
Epoch: 3 | Training Loss: 0.0032106723299940043 | Training accuracy : 0.8719912808119326
Validation loss: 0.821329 | Validation accuracy: 0.869145
Epoch: 4 | Training Loss: 0.0031993298365287897 | Training accuracy : 0.8758621394531181
Validation loss: 0.825614 | Validation accuracy: 0.871846
Epoch: 5 | Training Loss: 0.003193462780612663 | Training accuracy : 0.877651691479744
Validation loss: 0.869779 | Validation accuracy: 0.782653
Epoch: 6 | Training Loss: 0.0031892008534283455 | Training accuracy : 0.8794087230905173
Validation loss: 0.846814 | Vali

In [ ]:
if __name__ == "__main__":

  for e in range(35, 7, -1):
    batch_size = 256
    dropout_fc = 0.25 # 0.25
    hidden_neurons = 256
    out_channels = 24
    epochs = e # 14, 20, 23
    lr = 0.001 # 0.001
    sl_thresh = 0.5 # 0.5

    # For reproducibility
    seed_num = 42
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    g = torch.Generator()
    g.manual_seed(seed_num)

    train_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_tr_data, wafer_tr_param_data, wafer_tr_label), batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_val_data, wafer_val_param_data, wafer_val_label), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_test_data, wafer_test_param_data, wafer_test_label), batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pos_weight = torch.tensor([(cls_cnts[0]/cls_cnts[1]).item()]).to(device)  # Actual Training ratio of classes after splitting (764978/311269)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    model = AttentionBasedCNN(in_channels=1, numHeads=32, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    training(model, optimizer, criterion, device, sl_thresh=sl_thresh, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)



Epoch: 0 | Training Loss: 0.0033285464970343142 | Training accuracy : 0.8333765390286803
Validation loss: 0.835135 | Validation accuracy: 0.857237
Epoch: 1 | Training Loss: 0.0032390257631862663 | Training accuracy : 0.8625900931663456
Validation loss: 0.833323 | Validation accuracy: 0.850439
Epoch: 2 | Training Loss: 0.0032128735575648808 | Training accuracy : 0.8716586434154985
Validation loss: 0.830021 | Validation accuracy: 0.872356
Epoch: 3 | Training Loss: 0.0031985909859813242 | Training accuracy : 0.8766221880293278
Validation loss: 0.818072 | Validation accuracy: 0.879531
Epoch: 4 | Training Loss: 0.0031922945941060994 | Training accuracy : 0.8786189415626711
Validation loss: 0.825406 | Validation accuracy: 0.877232
Epoch: 5 | Training Loss: 0.003188045075355786 | Training accuracy : 0.8803155781154326
Validation loss: 0.820677 | Validation accuracy: 0.880267
Epoch: 6 | Training Loss: 0.003184775268795476 | Training accuracy : 0.8815801577147253
Validation loss: 0.814246 | Val

In [ ]:
# Epochs =34

Epoch: 0 | Training Loss: 0.0033433085268370602 | Training accuracy : 0.8292492336796293
Validation loss: 0.848780 | Validation accuracy: 0.844886
Epoch: 1 | Training Loss: 0.0032931851819797886 | Training accuracy : 0.8456516022808891
Validation loss: 0.837604 | Validation accuracy: 0.852688
Epoch: 2 | Training Loss: 0.0032750091108354135 | Training accuracy : 0.8515006313606449
Validation loss: 0.870248 | Validation accuracy: 0.831138
Epoch: 3 | Training Loss: 0.0032454317888693197 | Training accuracy : 0.8606481597625824
Validation loss: 0.903009 | Validation accuracy: 0.710695
Epoch: 4 | Training Loss: 0.0032134121521745587 | Training accuracy : 0.8710491179069488
Validation loss: 0.824259 | Validation accuracy: 0.867598
Epoch: 5 | Training Loss: 0.003200718930177907 | Training accuracy : 0.8752804885867278
Validation loss: 0.819646 | Validation accuracy: 0.879239
Epoch: 6 | Training Loss: 0.0031934446906563894 | Training accuracy : 0.8778486722843362
Validation loss: 0.816087 | Va

In [ ]:
# Epochs =23

Epoch: 0 | Training Loss: 0.0033433085268370602 | Training accuracy : 0.8292492336796293
Validation loss: 0.848780 | Validation accuracy: 0.844886
Epoch: 1 | Training Loss: 0.0032931851819797886 | Training accuracy : 0.8456516022808891
Validation loss: 0.837604 | Validation accuracy: 0.852688
Epoch: 2 | Training Loss: 0.0032750091108354135 | Training accuracy : 0.8515006313606449
Validation loss: 0.870248 | Validation accuracy: 0.831138
Epoch: 3 | Training Loss: 0.0032454317888693197 | Training accuracy : 0.8606481597625824
Validation loss: 0.903009 | Validation accuracy: 0.710695
Epoch: 4 | Training Loss: 0.0032134121521745587 | Training accuracy : 0.8710491179069488
Validation loss: 0.824259 | Validation accuracy: 0.867598
Epoch: 5 | Training Loss: 0.003200718930177907 | Training accuracy : 0.8752804885867278
Validation loss: 0.819646 | Validation accuracy: 0.879239
Epoch: 6 | Training Loss: 0.0031934446906563894 | Training accuracy : 0.8778486722843362
Validation loss: 0.816087 | Va

In [ ]:
# Epochs =21

Epoch: 0 | Training Loss: 0.0033433085268370602 | Training accuracy : 0.8292492336796293
Validation loss: 0.848780 | Validation accuracy: 0.844886
Epoch: 1 | Training Loss: 0.0032931851819797886 | Training accuracy : 0.8456516022808891
Validation loss: 0.837604 | Validation accuracy: 0.852688
Epoch: 2 | Training Loss: 0.0032750091108354135 | Training accuracy : 0.8515006313606449
Validation loss: 0.870248 | Validation accuracy: 0.831138
Epoch: 3 | Training Loss: 0.0032454317888693197 | Training accuracy : 0.8606481597625824
Validation loss: 0.903009 | Validation accuracy: 0.710695
Epoch: 4 | Training Loss: 0.0032134121521745587 | Training accuracy : 0.8710491179069488
Validation loss: 0.824259 | Validation accuracy: 0.867598
Epoch: 5 | Training Loss: 0.003200718930177907 | Training accuracy : 0.8752804885867278
Validation loss: 0.819646 | Validation accuracy: 0.879239
Epoch: 6 | Training Loss: 0.0031934446906563894 | Training accuracy : 0.8778486722843362
Validation loss: 0.816087 | Va

In [ ]:
# Epochs 20

Epoch: 0 | Training Loss: 0.0033433085268370602 | Training accuracy : 0.8292492336796293
Validation loss: 0.848780 | Validation accuracy: 0.844886
Epoch: 1 | Training Loss: 0.0032931851819797886 | Training accuracy : 0.8456516022808891
Validation loss: 0.837604 | Validation accuracy: 0.852688
Epoch: 2 | Training Loss: 0.0032750091108354135 | Training accuracy : 0.8515006313606449
Validation loss: 0.870248 | Validation accuracy: 0.831138
Epoch: 3 | Training Loss: 0.0032454317888693197 | Training accuracy : 0.8606481597625824
Validation loss: 0.903009 | Validation accuracy: 0.710695
Epoch: 4 | Training Loss: 0.0032134121521745587 | Training accuracy : 0.8710491179069488
Validation loss: 0.824259 | Validation accuracy: 0.867598
Epoch: 5 | Training Loss: 0.003200718930177907 | Training accuracy : 0.8752804885867278
Validation loss: 0.819646 | Validation accuracy: 0.879239
Epoch: 6 | Training Loss: 0.0031934446906563894 | Training accuracy : 0.8778486722843362
Validation loss: 0.816087 | Va

In [ ]:
# epochs =11

Epoch: 0 | Training Loss: 0.0033285464970343142 | Training accuracy : 0.8333765390286803
Validation loss: 0.835135 | Validation accuracy: 0.857237
Epoch: 1 | Training Loss: 0.0032390257631862663 | Training accuracy : 0.8625900931663456
Validation loss: 0.833323 | Validation accuracy: 0.850439
Epoch: 2 | Training Loss: 0.0032128735575648808 | Training accuracy : 0.8716586434154985
Validation loss: 0.830021 | Validation accuracy: 0.872356
Epoch: 3 | Training Loss: 0.0031985909859813242 | Training accuracy : 0.8766221880293278
Validation loss: 0.818072 | Validation accuracy: 0.879531
Epoch: 4 | Training Loss: 0.0031922945941060994 | Training accuracy : 0.8786189415626711
Validation loss: 0.825406 | Validation accuracy: 0.877232
Epoch: 5 | Training Loss: 0.003188045075355786 | Training accuracy : 0.8803155781154326
Validation loss: 0.820677 | Validation accuracy: 0.880267
Epoch: 6 | Training Loss: 0.003184775268795476 | Training accuracy : 0.8815801577147253
Validation loss: 0.814246 | Val

In [ ]:
# epochs = 12

Epoch: 0 | Training Loss: 0.0033285464970343142 | Training accuracy : 0.8333765390286803
Validation loss: 0.835135 | Validation accuracy: 0.857237
Epoch: 1 | Training Loss: 0.0032390257631862663 | Training accuracy : 0.8625900931663456
Validation loss: 0.833323 | Validation accuracy: 0.850439
Epoch: 2 | Training Loss: 0.0032128735575648808 | Training accuracy : 0.8716586434154985
Validation loss: 0.830021 | Validation accuracy: 0.872356
Epoch: 3 | Training Loss: 0.0031985909859813242 | Training accuracy : 0.8766221880293278
Validation loss: 0.818072 | Validation accuracy: 0.879531
Epoch: 4 | Training Loss: 0.0031922945941060994 | Training accuracy : 0.8786189415626711
Validation loss: 0.825406 | Validation accuracy: 0.877232
Epoch: 5 | Training Loss: 0.003188045075355786 | Training accuracy : 0.8803155781154326
Validation loss: 0.820677 | Validation accuracy: 0.880267
Epoch: 6 | Training Loss: 0.003184775268795476 | Training accuracy : 0.8815801577147253
Validation loss: 0.814246 | Val

In [ ]:
# Epochs =26

Epoch: 0 | Training Loss: 0.0032619682683201847 | Training accuracy : 0.8563879852859055
Validation loss: 0.829648 | Validation accuracy: 0.851342
Epoch: 1 | Training Loss: 0.0032043257697809157 | Training accuracy : 0.8753650416679443
Validation loss: 0.842413 | Validation accuracy: 0.822684
Epoch: 2 | Training Loss: 0.0031832761673443017 | Training accuracy : 0.8817901466856586
Validation loss: 0.809264 | Validation accuracy: 0.889282
Epoch: 3 | Training Loss: 0.0031745756118068983 | Training accuracy : 0.8843648344664375
Validation loss: 0.814372 | Validation accuracy: 0.881856
Epoch: 4 | Training Loss: 0.0031718572942113954 | Training accuracy : 0.8853943379168536
Validation loss: 0.842347 | Validation accuracy: 0.817750
Epoch: 5 | Training Loss: 0.003167567862196703 | Training accuracy : 0.8866552008971919
Validation loss: 0.808781 | Validation accuracy: 0.888212
Epoch: 6 | Training Loss: 0.0031641499609555133 | Training accuracy : 0.8877776198214722
Validation loss: 0.812896 | Va

In [ ]:
# pos_weight = torch.tensor([764978/311269]).to(device), Epochs = 24

Epoch: 0 | Training Loss: 0.0032619682683201847 | Training accuracy : 0.8563879852859055
Validation loss: 0.829648 | Validation accuracy: 0.851342
Epoch: 1 | Training Loss: 0.0032043257697809157 | Training accuracy : 0.8753650416679443
Validation loss: 0.842413 | Validation accuracy: 0.822684
Epoch: 2 | Training Loss: 0.0031832761673443017 | Training accuracy : 0.8817901466856586
Validation loss: 0.809264 | Validation accuracy: 0.889282
Epoch: 3 | Training Loss: 0.0031745756118068983 | Training accuracy : 0.8843648344664375
Validation loss: 0.814372 | Validation accuracy: 0.881856
Epoch: 4 | Training Loss: 0.0031718572942113954 | Training accuracy : 0.8853943379168536
Validation loss: 0.842347 | Validation accuracy: 0.817750
Epoch: 5 | Training Loss: 0.003167567862196703 | Training accuracy : 0.8866552008971919
Validation loss: 0.808781 | Validation accuracy: 0.888212
Epoch: 6 | Training Loss: 0.0031641499609555133 | Training accuracy : 0.8877776198214722
Validation loss: 0.812896 | Va

In [ ]:
# pos_weight = torch.tensor([764978/311269]).to(device), Epochs = 21

Epoch: 0 | Training Loss: 0.0032619682683201847 | Training accuracy : 0.8563879852859055
Validation loss: 0.829648 | Validation accuracy: 0.851342
Epoch: 1 | Training Loss: 0.0032043257697809157 | Training accuracy : 0.8753650416679443
Validation loss: 0.842413 | Validation accuracy: 0.822684
Epoch: 2 | Training Loss: 0.0031832761673443017 | Training accuracy : 0.8817901466856586
Validation loss: 0.809264 | Validation accuracy: 0.889282
Epoch: 3 | Training Loss: 0.0031745756118068983 | Training accuracy : 0.8843648344664375
Validation loss: 0.814372 | Validation accuracy: 0.881856
Epoch: 4 | Training Loss: 0.0031718572942113954 | Training accuracy : 0.8853943379168536
Validation loss: 0.842347 | Validation accuracy: 0.817750
Epoch: 5 | Training Loss: 0.003167567862196703 | Training accuracy : 0.8866552008971919
Validation loss: 0.808781 | Validation accuracy: 0.888212
Epoch: 6 | Training Loss: 0.0031641499609555133 | Training accuracy : 0.8877776198214722
Validation loss: 0.812896 | Va

In [ ]:
# pos_weight = torch.tensor([764978/311269]).to(device), Epochs = 13

Epoch: 0 | Training Loss: 0.0032619682683201847 | Training accuracy : 0.8563879852859055
Validation loss: 0.829648 | Validation accuracy: 0.851342
Epoch: 1 | Training Loss: 0.0032043257697809157 | Training accuracy : 0.8753650416679443
Validation loss: 0.842413 | Validation accuracy: 0.822684
Epoch: 2 | Training Loss: 0.0031832761673443017 | Training accuracy : 0.8817901466856586
Validation loss: 0.809264 | Validation accuracy: 0.889282
Epoch: 3 | Training Loss: 0.0031745756118068983 | Training accuracy : 0.8843648344664375
Validation loss: 0.814372 | Validation accuracy: 0.881856
Epoch: 4 | Training Loss: 0.0031718572942113954 | Training accuracy : 0.8853943379168536
Validation loss: 0.842347 | Validation accuracy: 0.817750
Epoch: 5 | Training Loss: 0.003167567862196703 | Training accuracy : 0.8866552008971919
Validation loss: 0.808781 | Validation accuracy: 0.888212
Epoch: 6 | Training Loss: 0.0031641499609555133 | Training accuracy : 0.8877776198214722
Validation loss: 0.812896 | Va

In [ ]:
# pos_weight = torch.tensor([764978/311269]).to(device), Epochs = 12

Epoch: 0 | Training Loss: 0.0032619682683201847 | Training accuracy : 0.8563879852859055
Validation loss: 0.829648 | Validation accuracy: 0.851342
Epoch: 1 | Training Loss: 0.0032043257697809157 | Training accuracy : 0.8753650416679443
Validation loss: 0.842413 | Validation accuracy: 0.822684
Epoch: 2 | Training Loss: 0.0031832761673443017 | Training accuracy : 0.8817901466856586
Validation loss: 0.809264 | Validation accuracy: 0.889282
Epoch: 3 | Training Loss: 0.0031745756118068983 | Training accuracy : 0.8843648344664375
Validation loss: 0.814372 | Validation accuracy: 0.881856
Epoch: 4 | Training Loss: 0.0031718572942113954 | Training accuracy : 0.8853943379168536
Validation loss: 0.842347 | Validation accuracy: 0.817750
Epoch: 5 | Training Loss: 0.003167567862196703 | Training accuracy : 0.8866552008971919
Validation loss: 0.808781 | Validation accuracy: 0.888212
Epoch: 6 | Training Loss: 0.0031641499609555133 | Training accuracy : 0.8877776198214722
Validation loss: 0.812896 | Va

In [ ]:
# pos_weight = torch.tensor([849976/345854]).to(device)

Epoch: 0 | Training Loss: 0.0032635117773353994 | Training accuracy : 0.8558481463827542
Validation loss: 0.824395 | Validation accuracy: 0.873209
Epoch: 1 | Training Loss: 0.003208778963916023 | Training accuracy : 0.8739294975967413
Validation loss: 0.851500 | Validation accuracy: 0.849929
Epoch: 2 | Training Loss: 0.0031863918923808374 | Training accuracy : 0.8808795750417887
Validation loss: 0.815961 | Validation accuracy: 0.885209
Epoch: 3 | Training Loss: 0.003176090244624186 | Training accuracy : 0.8842328944935502
Validation loss: 0.811278 | Validation accuracy: 0.888136
Epoch: 4 | Training Loss: 0.003172144358955791 | Training accuracy : 0.8852159402070343
Validation loss: 0.898219 | Validation accuracy: 0.720211
Epoch: 5 | Training Loss: 0.0031701883185229675 | Training accuracy : 0.8856916674332194
Validation loss: 0.808787 | Validation accuracy: 0.887693
Epoch: 6 | Training Loss: 0.0031670557766501524 | Training accuracy : 0.8868605440944318
Validation loss: 0.811391 | Vali

In [ ]:
# pos_weight = torch.tensor([850231/345599]).to(device)
# Epoch: 11 | Training Loss: 0.0031572366311809204 | Training accuracy : 0.8902157218556707
# Validation loss: 0.807439 | Validation accuracy: 0.891598

# Confusion Matrix :
# [[203114   8499]
#  [ 24338  61832]]

# Classification Report :
#               precision    recall  f1-score   support

#          0.0       0.89      0.96      0.93    211613
#          1.0       0.88      0.72      0.79     86170

#     accuracy                           0.89    297783
#    macro avg       0.89      0.84      0.86    297783
# weighted avg       0.89      0.89      0.89    297783


# *************************
# Covered Test Escapes : 61.89389389388707
# Yield Loss : 0.028540917379434018

In [ ]:
# sl_thresh >= 0.9

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8575243415312656
Validation loss: 0.821269 | Validation accuracy: 0.874271
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8760934989830401
Validation loss: 0.816096 | Validation accuracy: 0.880936
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8802421748910798
Validation loss: 0.814334 | Validation accuracy: 0.884440
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8828623912540523
Validation loss: 0.812113 | Validation accuracy: 0.885987
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8856061851972642
Validation loss: 0.812230 | Validation accuracy: 0.885795
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.8876447506938463
Validation loss: 0.812169 | Validation accuracy: 0.886430
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8885423141713752
Validation loss: 0.809365 | Valida

In [ ]:
# sl_thresh >= 0.8

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8581924037883497
Validation loss: 0.821269 | Validation accuracy: 0.875476
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8760488995555853
Validation loss: 0.816096 | Validation accuracy: 0.881622
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8800563439433513
Validation loss: 0.814334 | Validation accuracy: 0.884532
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.882603157081971
Validation loss: 0.812113 | Validation accuracy: 0.885778
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.88538318805999
Validation loss: 0.812230 | Validation accuracy: 0.885335
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.8873892331407196
Validation loss: 0.812169 | Validation accuracy: 0.885812
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8882700718329528
Validation loss: 0.809365 | Validatio

In [ ]:
# sl_thresh >= 0.7

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8579638317226436
Validation loss: 0.821269 | Validation accuracy: 0.876178
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.875620559221071
Validation loss: 0.816096 | Validation accuracy: 0.881848
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8794449601253244
Validation loss: 0.814334 | Validation accuracy: 0.884607
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8820493808577399
Validation loss: 0.812113 | Validation accuracy: 0.884883
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8849241856191005
Validation loss: 0.812230 | Validation accuracy: 0.884808
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.8870556665895468
Validation loss: 0.812169 | Validation accuracy: 0.885218
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8877488160245743
Validation loss: 0.809365 | Validat

In [ ]:
# sl_thresh >= 0.6

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8574091263436739
Validation loss: 0.821269 | Validation accuracy: 0.875802
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8750742162347491
Validation loss: 0.816096 | Validation accuracy: 0.881664
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8789432165664574
Validation loss: 0.814334 | Validation accuracy: 0.884624
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8814547218250086
Validation loss: 0.812113 | Validation accuracy: 0.884315
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8843555429190512
Validation loss: 0.812230 | Validation accuracy: 0.884515
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.8865464897927706
Validation loss: 0.812169 | Validation accuracy: 0.885059
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8872461433109686
Validation loss: 0.809365 | Valida

In [ ]:
# sl_thresh >= 0.5

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8564000642975079
Validation loss: 0.821269 | Validation accuracy: 0.876120
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8744015082039718
Validation loss: 0.816096 | Validation accuracy: 0.881262
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8782082551681909
Validation loss: 0.814334 | Validation accuracy: 0.884206
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8806770193087646
Validation loss: 0.812113 | Validation accuracy: 0.883428
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8835146578805795
Validation loss: 0.812230 | Validation accuracy: 0.883846
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.885845907119834
Validation loss: 0.812169 | Validation accuracy: 0.884624
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8866579883614077
Validation loss: 0.809365 | Validat

In [ ]:
# Epochs =13

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8564000642975079
Validation loss: 0.821269 | Validation accuracy: 0.876120
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8744015082039718
Validation loss: 0.816096 | Validation accuracy: 0.881262
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8782082551681909
Validation loss: 0.814334 | Validation accuracy: 0.884206
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8806770193087646
Validation loss: 0.812113 | Validation accuracy: 0.883428
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8835146578805795
Validation loss: 0.812230 | Validation accuracy: 0.883846
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.885845907119834
Validation loss: 0.812169 | Validation accuracy: 0.884624
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8866579883614077
Validation loss: 0.809365 | Validat

In [ ]:
Epochs=12

Epoch: 0 | Training Loss: 0.0032623445752981684 | Training accuracy : 0.8564000642975079
Validation loss: 0.821269 | Validation accuracy: 0.876120
Epoch: 1 | Training Loss: 0.003207787858498324 | Training accuracy : 0.8744015082039718
Validation loss: 0.816096 | Validation accuracy: 0.881262
Epoch: 2 | Training Loss: 0.003194146611641393 | Training accuracy : 0.8782082551681909
Validation loss: 0.814334 | Validation accuracy: 0.884206
Epoch: 3 | Training Loss: 0.0031860027504171623 | Training accuracy : 0.8806770193087646
Validation loss: 0.812113 | Validation accuracy: 0.883428
Epoch: 4 | Training Loss: 0.003177091175019239 | Training accuracy : 0.8835146578805795
Validation loss: 0.812230 | Validation accuracy: 0.883846
Epoch: 5 | Training Loss: 0.003170768389496846 | Training accuracy : 0.885845907119834
Validation loss: 0.812169 | Validation accuracy: 0.884624
Epoch: 6 | Training Loss: 0.003168707576350432 | Training accuracy : 0.8866579883614077
Validation loss: 0.809365 | Validat

Epoch: 0 | Training Loss: 0.0032614444637893785 | Training accuracy : 0.8568971620826817
Validation loss: 0.819716 | Validation accuracy: 0.873903
Epoch: 1 | Training Loss: 0.0031959907759624767 | Training accuracy : 0.8785408925646251
Validation loss: 0.812421 | Validation accuracy: 0.886389
Epoch: 2 | Training Loss: 0.003178917983859313 | Training accuracy : 0.8838157040158997
Validation loss: 0.813348 | Validation accuracy: 0.886522
Epoch: 3 | Training Loss: 0.00317220795104473 | Training accuracy : 0.8856331306846849
Validation loss: 0.809998 | Validation accuracy: 0.889817
Epoch: 4 | Training Loss: 0.0031681805722942953 | Training accuracy : 0.8868958519745003
Validation loss: 0.808139 | Validation accuracy: 0.890168
Epoch: 5 | Training Loss: 0.003163416893666938 | Training accuracy : 0.8883174587246236
Validation loss: 0.818772 | Validation accuracy: 0.867013
Epoch: 6 | Training Loss: 0.0031607886735129116 | Training accuracy : 0.8892782047243802
Validation loss: 0.809301 | Valid

Epoch: 0 | Training Loss: 0.003261364989378984 | Training accuracy : 0.8567522139434535
Validation loss: 0.819576 | Validation accuracy: 0.875944
Epoch: 1 | Training Loss: 0.003191518086499033 | Training accuracy : 0.8800795728118174
Validation loss: 0.815728 | Validation accuracy: 0.884089
Epoch: 2 | Training Loss: 0.003176619514078502 | Training accuracy : 0.8845320823193932
Validation loss: 0.810496 | Validation accuracy: 0.884181
Epoch: 3 | Training Loss: 0.0031707176883293553 | Training accuracy : 0.8861014246729607
Validation loss: 0.809663 | Validation accuracy: 0.887476
Epoch: 4 | Training Loss: 0.0031656667980847026 | Training accuracy : 0.8878593854384728
Validation loss: 0.812524 | Validation accuracy: 0.887685
Epoch: 5 | Training Loss: 0.0031636328703770673 | Training accuracy : 0.8882858674635098
Validation loss: 0.808307 | Validation accuracy: 0.891465
Epoch: 6 | Training Loss: 0.0031598122034923303 | Training accuracy : 0.8896628747861782
Validation loss: 0.808559 | Vali

In [ ]:


dropout_fc = 0.1
hidden_neurons = 256
out_channels = 32
epochs = 20
lr = 0.001
network = AttentionBasedCNN(in_channels=1, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)
training(network, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)

Epoch: 0 | Training Loss: 0.0035794402444889557 | Training accuracy : 0.7606153606003083
Validation loss: 0.913404 | Validation accuracy: 0.765728
Epoch: 1 | Training Loss: 0.0035722090333550733 | Training accuracy : 0.7638423150076145
Validation loss: 0.912849 | Validation accuracy: 0.765251
Epoch: 2 | Training Loss: 0.0035699788951940857 | Training accuracy : 0.7650437120846795
Validation loss: 0.913001 | Validation accuracy: 0.769215
Epoch: 3 | Training Loss: 0.003568384621875971 | Training accuracy : 0.7655166518466486
Validation loss: 0.913116 | Validation accuracy: 0.763077
Epoch: 4 | Training Loss: 0.0035670257206319228 | Training accuracy : 0.7662506840901763
Validation loss: 0.912998 | Validation accuracy: 0.764448
Epoch: 5 | Training Loss: 0.0035657669714202205 | Training accuracy : 0.7668908717051012
Validation loss: 0.912675 | Validation accuracy: 0.767785
Epoch: 6 | Training Loss: 0.0035639366657645726 | Training accuracy : 0.7672978414806266
Validation loss: 0.912744 | Va

In [ ]:
# Augmentation with applied 850231/345599
dropout_fc = 0.1
hidden_neurons = 256
out_channels = 32
epochs = 20
lr = 0.001
network = AttentionBasedCNN(in_channels=1, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)
training(network, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)

Epoch: 0 | Training Loss: 0.0036791002898043743 | Training accuracy : 0.7384212778886773
Validation loss: 0.938760 | Validation accuracy: 0.743465
Epoch: 1 | Training Loss: 0.0036566275447991736 | Training accuracy : 0.7481016587004179
Validation loss: 0.934752 | Validation accuracy: 0.750628
Epoch: 2 | Training Loss: 0.003643862849989939 | Training accuracy : 0.75261546704652
Validation loss: 0.933012 | Validation accuracy: 0.751727
Epoch: 3 | Training Loss: 0.0036374376113218402 | Training accuracy : 0.7549573154993907
Validation loss: 0.931525 | Validation accuracy: 0.756032
Epoch: 4 | Training Loss: 0.0036314374301864025 | Training accuracy : 0.7570116602359491
Validation loss: 0.930568 | Validation accuracy: 0.756473
Epoch: 5 | Training Loss: 0.003627470956968007 | Training accuracy : 0.7580597419654146
Validation loss: 0.930168 | Validation accuracy: 0.753956
Epoch: 6 | Training Loss: 0.0036243236007308687 | Training accuracy : 0.7591783927888841
Validation loss: 0.930359 | Valid

In [ ]:
42380/(211853+85930)

0.1423183996400063

In [ ]:
# Augmentation
dropout_fc = 0.1
hidden_neurons = 256
out_channels = 32
epochs = 20
lr = 0.001
network = AttentionBasedCNN(in_channels=1, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)
training(network, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)

Epoch: 0 | Training Loss: 0.0023788236717381056 | Training accuracy : 0.7384003685274909
Validation loss: 0.607083 | Validation accuracy: 0.740554
Epoch: 1 | Training Loss: 0.0023673403382775313 | Training accuracy : 0.7448901115057027
Validation loss: 0.605202 | Validation accuracy: 0.746917
Epoch: 2 | Training Loss: 0.002361453803204239 | Training accuracy : 0.7484852147946799
Validation loss: 0.604472 | Validation accuracy: 0.749493
Epoch: 3 | Training Loss: 0.0023566563144310484 | Training accuracy : 0.7511413570828828
Validation loss: 0.603152 | Validation accuracy: 0.750716
Epoch: 4 | Training Loss: 0.002353507206292543 | Training accuracy : 0.7532303329489061
Validation loss: 0.602666 | Validation accuracy: 0.753021
Epoch: 5 | Training Loss: 0.00235186837702941 | Training accuracy : 0.7545476227036457
Validation loss: 0.602597 | Validation accuracy: 0.754362
Epoch: 6 | Training Loss: 0.0023504357728306845 | Training accuracy : 0.7553944518316927
Validation loss: 0.602270 | Valid

In [ ]:
if __name__ == "__main__":

  # for e in range(41, 7, -1):
    batch_size = 256
    dropout_fc = 0.35 # 0.25
    hidden_neurons = 256
    out_channels = 24
    epochs = 24 # 14, 20, 23
    lr = 0.001 # 0.001
    sl_thresh = 0.5 # 0.5

    # For reproducibility
    seed_num = 42
    torch.manual_seed(seed_num)
    torch.cuda.manual_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

    g = torch.Generator()
    g.manual_seed(seed_num)

    train_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_tr_data, wafer_tr_param_data, wafer_tr_label), batch_size=batch_size, shuffle=True, worker_init_fn=seed_worker, generator=g)
    val_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_val_data, wafer_val_param_data, wafer_val_label), batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset = Wafer_receptive_field_v1(wafer_test_data, wafer_test_param_data, wafer_test_label), batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pos_weight = torch.tensor([(cls_cnts[0]/cls_cnts[1]).item()]).to(device)  # Actual Training ratio of classes after splitting (764978/311269)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    model = AttentionBasedCNN(in_channels=1, numHeads=32, numCls=1, out_channels=out_channels, dropout_fc=dropout_fc, hidden_neurons=hidden_neurons).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    training(model, optimizer, criterion, device, sl_thresh=sl_thresh, epochs=epochs, lr=lr, dataloaders=(train_loader, val_loader, test_loader), dropout_fc=dropout_fc, hidden_neurons=hidden_neurons)



Epoch: 0 | Training Loss: 0.00334632878410096 | Training accuracy : 0.8284668853896922
Validation loss: 0.849972 | Validation accuracy: 0.835119
Epoch: 1 | Training Loss: 0.003277281447430311 | Training accuracy : 0.8502602097845569
Validation loss: 0.847792 | Validation accuracy: 0.821622
Epoch: 2 | Training Loss: 0.003231749665046905 | Training accuracy : 0.8650319118194987
Validation loss: 0.829828 | Validation accuracy: 0.868443
Epoch: 3 | Training Loss: 0.0032118053433762743 | Training accuracy : 0.871587098500623
Validation loss: 0.857958 | Validation accuracy: 0.800582
Epoch: 4 | Training Loss: 0.0032013922087875263 | Training accuracy : 0.8752154477550228
Validation loss: 0.820501 | Validation accuracy: 0.871470
Epoch: 5 | Training Loss: 0.003193085116500285 | Training accuracy : 0.8780149909825533
Validation loss: 0.823770 | Validation accuracy: 0.876253
Epoch: 6 | Training Loss: 0.0031881964259408086 | Training accuracy : 0.8799030334114752
Validation loss: 0.814308 | Validat